# 18.303 pset 1 solutions

These are the solutions for problem 2, the computational portion of the problem set.  For the solutions to the other problems, see the PDF file posted to the web page.

## Problem 2

We begin by making a random real-symmetric matrix, with the `randn` function to generate Gaussian-distributed entries:

In [1]:
A = full(Symmetric(randn(5,5)))

5x5 Array{Float64,2}:
  0.768735  -1.95355    0.959372  -0.78639    0.675625
 -1.95355   -0.87322   -0.125788  -0.901366  -1.06813 
  0.959372  -0.125788  -0.357172   0.201748  -0.347005
 -0.78639   -0.901366   0.201748   0.974272   0.222174
  0.675625  -1.06813   -0.347005   0.222174  -1.66626 

### part (a): 10 points

Compare the following:

In [2]:
eigvals(A)

5-element Array{Float64,1}:
 -2.79271 
 -2.15493 
 -0.210511
  1.41744 
  2.58706 

In [3]:
Q,R = qr(A)
eigvals(R*Q)

5-element Array{Float64,1}:
 -2.79271 
 -2.15493 
  2.58706 
  1.41744 
 -0.210511

They look the same (although the order is somewhat random, as is typical for numerical eigensolvers).  Let's check this quantitatively:

In [4]:
norm(sort(eigvals(A)) - sort(eigvals(R*Q)))

1.9984014443252818e-15

The difference is essentially zero; the small nonzero value on the order of $10^{-15}$ is due to roundoff errors, thanks to the computer's finite-precision [floating-point arithmetic](https://en.wikipedia.org/wiki/Floating_point).

**Proof:** If $A = QR$, then $R = Q^{-1} A$.   Therefore, $RQ =Q^{-1} A Q$, which is a [similar matrix](https://en.wikipedia.org/wiki/Matrix_similarity) to $A$ by inspection.

(Furthermore, since $Q^* = Q^{-1}$, we have $RQ = Q^* A Q$, so if $A=A^*$ then this is still true of $RQ$: this transformation preserves Hermitian-ness.  But I didn't ask you to prove that.)

### part (b): 10 points

As suggested, let us try doing this transformation repeatedly:

In [5]:
B = A
for i = 1:100000
    Q,R = qr(B)
    B = R*Q
end
B

5x5 Array{Float64,2}:
 -2.79271        4.49107e-16    7.18755e-16   5.54666e-17   1.27451e-16
 -1.5316e-322    2.58706        1.15245e-16  -2.97053e-16   8.91943e-17
  4.44659e-323  -6.42285e-323  -2.15493       2.38545e-16   2.56529e-16
  1.4822e-323    1.4822e-323    1.4822e-323   1.41744       5.42042e-17
  0.0            0.0            0.0           0.0          -0.210511   

All of the off-diagonal elements of $B$ are tiny: $B$ is converging to a **diagonal matrix**.   Furthermore, the **diagonal elements are the eigenvalues** of $A$ in descending order by magnitude!

In [6]:
diag(B)

5-element Array{Float64,1}:
 -2.79271 
  2.58706 
 -2.15493 
  1.41744 
 -0.210511

Proving why this works is a little bit tricky, although not terrible (it takes me a lecture or two in 18.335), but this amazing fact is the foundation of the [QR algorithm](https://en.wikipedia.org/wiki/QR_algorithm) for computing eigenvalues.   This (plus some additional tricks!) is essentially what the `eigvals` function is doing in Julia (or any other numerical linear-algebra library).

In 18.06, you learned to compute eigenvalues by finding the roots of a characteristic polynomial $\det(A - \lambda I) = 0$. Unfortunately, this approach (implemented in the most straightforward way, at least) is nearly useless for any eigenproblem bigger than $2 \times 2$.